In [1]:
from awsglue.context import GlueContext
from pyspark.context import SparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
glue_context = GlueContext(SparkContext.getOrCreate())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Get data from the s3 bucket with jsonPath formatting for [] json.
data = glue_context.create_dynamic_frame.from_options(
    connection_type="s3",
    connection_options={"paths": ["s3://eng-dados-098472938/"]},
    format="json",
    format_options={"jsonPath": "$[*]"}
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# create a view named as "source" to query with spark sql.
read_data = data.toDF().createOrReplaceTempView("source")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Calculate total_liquido with spark sql.
sql_data = spark.sql(
"""WITH clean_data AS (
    SELECT
        total_bruto,
        CASE
            WHEN desconto_percentual IS NULL THEN 0
            ELSE desconto_percentual
        END AS desconto_percentual
    FROM source)
SELECT
    ROUND(
        SUM(
            (total_bruto * (100-desconto_percentual)/100)
            ), 2) AS total
FROM clean_data
""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
sql_data.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|   total|
+--------+
|59973.46|
+--------+